#TODO VH setup

# Setup

In [1]:
!git clone https://github.com/szy4806384/lm_on_decison_making
%cd ./lm_on_decison_making 
!pip install transformers

Cloning into 'lm_on_decison_making'...
remote: Enumerating objects: 200, done.
remote: Counting objects: 100% (65/65), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 200 (delta 28), reused 31 (delta 12), pack-reused 135
Receiving objects: 100% (200/200), 30.89 MiB | 11.77 MiB/s, done.
Resolving deltas: 100% (93/93), done.
/content/lm_on_decison_making
     |████████████████████████████████| 4.0 MB 12.2 MB/s 
     |████████████████████████████████| 596 kB 41.5 MB/s 
     |████████████████████████████████| 895 kB 45.9 MB/s 
     |████████████████████████████████| 6.6 MB 9.3 MB/s 
     |████████████████████████████████| 77 kB 3.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
import os
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader, IterableDataset
from transformers import GPT2Tokenizer, GPT2Model
from train_utils import *

drive.mount('/content/drive')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mounted at /content/drive


In [3]:
#define tokenizer and embedders
pretrained_lm = 'gpt2'
tokenizer = get_tokenizer(pretrained_lm)
tokenizer.pad_token = tokenizer.eos_token
word_embeddings = get_word_embeddings(pretrained_lm).to(device)

Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/523M [00:00<?, ?B/s]

#Test

In [4]:
model = LMDecisionMaker()

In [ ]:
model_path = '/content/drive/MyDrive/models/model_7.p'
model.load_state_dict(torch.load(model_path))
model.eval()

In [6]:
data_path = '/content/drive/MyDrive/Dataset/lm_vh/training_data'
data_loader = VHDataloader(data_path, batch_size = 1, mini_batch = True)

In [7]:
for ind, data in enumerate(data_loader):
    sample = preprocess(data, tokenizer, word_embeddings)
    prediction = model(**sample, mode='test')
    break

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2269: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
prediction

In [ ]:
sample

In [13]:
def get_key_action(raw_goals):
    info = " ".join(raw_goals[1:-1].split()[1:])
    raw_goals = info[1:-1].split(') (')
    key_actions = set()
    key_action = ''
    for raw_sub_goals in raw_goals:
        split_goal = raw_sub_goals.split(' ')
        if split_goal[0] == 'inside':
            key_action = '[putin] <{}> <{}>'.format(split_goal[1].split('_')[0], split_goal[2].split('_')[0])
        elif split_goal[0] == 'ontop':
            key_action = '[put] <{}> <{}>'.format(split_goal[1].split('_')[0], split_goal[2].split('_')[0])
        elif split_goal[0] == 'grabbed':
            key_action = '[grab] <{}>'.format(split_goal[2].split('_')[0])
        key_actions.add(key_action)

    return key_actions

goals = '(and  (ontop clothespants_387  clothespile_413) (inside plate_195 kitchencabinet_235) (grabbed character_1 towel_68) (grabbed character_2 cereal_335))'
get_key_action(goals)

{'[grab] <cereal>',
 '[grab] <towel>',
 '[put] <clothespants> <clothespile>',
 '[putin] <plate> <kitchencabinet>'}

In [ ]:
action_space = {0: 'standup', 1: 'walk', 2:'sit', 3:'grab', 4:'open', 5:'close', 6:'switchon', 7:'switchoff', 8:'put', 9: 'putin'}

def judge_standup(sitting_edges):
    cond1 = len(sitting_edges) > 1
    return True if cond1 else False

def judge_walk(sitting_edges, inside_edges, hold_edges, obj, index):
    cond1 = len(sitting_edges) == 0
    cond2 = index not in inside_edges
    cond3 = obj in room
    cond4 = index not in hold_edges
    return True if cond1 and (cond2 or cond3) and cond4 else False

def judge_sit(sitting_edges, close_edges, obj, index):
    cond1 = len(sitting_edges) == 0
    cond2 = index in close_edges
    cond3 = 'SITTABLE' in properties[obj] if obj in properties else False
    return True if cond1 and cond2 and cond3 else False

def judge_grab(close_edges, inside_edges, hold_edges, obj, index):
    cond1 = 'GRABBABLE' in properties[obj] if obj in properties else False
    cond2 = obj != 'water'
    cond3 = index in close_edges
    cond4 = index not in inside_edges
    cond5 = index not in hold_edges
    cond6 = len(hold_edges) < 2
    return True if cond1 and cond2 and cond3 and cond4 and cond5 and cond6 else False

def judge_open(observations, close_edges, inside_edges, hold_edges, obj, index):
    cond1 = 'CAN_OPEN' in properties[obj] if obj in properties else False
    cond2 = observations[index]['state'][1] = 1
    cond3 = index in close_edges
    cond4 = index not in inside_edges
    cond5 = index not in hold_edges
    cond6 = len(hold_edges) < 2
    return True if cond1 and cond2 and cond3 and cond4 and cond5 and cond6 else False

def judge_close(observations, close_edges, inside_edges, hold_edges, obj, index):
    cond1 = 'CAN_OPEN' in properties[obj] if obj in properties else False
    cond2 = observations[index]['state'][0] = 1
    cond3 = index in close_edges
    cond4 = index not in inside_edges
    cond5 = index not in hold_edges
    cond6 = len(hold_edges) < 2
    return True if cond1 and cond2 and cond3 and cond4 and cond5 and cond6 else False

def judge_switchon(observations, close_edges, obj, index):
    cond1 = 'HAS_SWITCH' in properties[obj] if obj in properties else False
    cond2 = observations[index]['state'][3] == 1
    cond3 = index in close_edges
    return True if cond1 and cond2 and cond3 else False

def judge_switchoff(observations, close_edges, obj, index):
    cond1 = 'HAS_SWITCH' in properties[obj] if obj in properties else False
    cond2 = observations[index]['state'][2] == 1
    cond3 = index in close_edges
    return True if cond1 and cond2 and cond3 else False

def judge_put(hold_edges, close_edges, index1, index2):
    cond1 = index1 in hold_edges
    cond2 = index2 in close_edges
    return True if cond1 and cond2 else False

def judge_putin(observations, hold_edges, close_edges, index1, index2):
    cond1 = index1 in hold_edges
    cond2 = index2 in close_edges
    cond3 = observations[index2]['state'][1] != 1
    return True if cond1 and cond2 and cond3 else False

def judge_0(relationships):
    sitting_edges = relationships['sitting']
    return judge_standup(sitting_edges)

def judge_1(action, obj, index, relationships, observations):
    sitting_edges = relationships['sitting']
    close_edges = relationships['close']
    inside_edges = relationships['inside']
    hold_edges = relationships['hold']
    if action == 'walk':
        return judge_walk(sitting_edges, inside_edges, hold_edges, obj, index)
    elif action == 'sit':
        return judge_sit(sitting_edges, close_edges, obj, index)
    elif action == 'grab':
        return judge_grab(close_edges, inside_edges, hold_edges, obj, index)
    elif action == 'open':
        return judge_open(observations, close_edges, inside_edges, hold_edges, obj, index)
    elif action == 'close':
        judge_close(observations, close_edges, inside_edges, hold_edges, obj, index)
    elif action == 'switchon':
        return judge_switchon(observations, close_edges, obj, index)
    elif action == 'switchoff':
        return judge_switchoff(observations, close_edges, obj, index)

def judge_2(action, index1, index2, relationships, observations):
    close_edges = relationships['close']
    hold_edges = relationships['hold']
    if action == 'put':
        return judge_put(hold_edges, close_edges, index1, index2)
    elif action == 'putin':
        judge_putin(observations, hold_edges, close_edges, index1, index2)

In [ ]:
action_index = {'walk': 0, 'grab': 1, 'open': 2, 'close': 3, 'switchon': 4, 'switchoff': 5, 'put': 6, 'putin': 7}
def parse_prediction(data, prediction):
    action_prediction = prediction['action']

    for action_id in action_prediction:
        if int(action_id.cpu().numpy()) == 0 and judge_0(relationships):
            return '[{}]'.format(action_space[int(action_id.cpu().numpy())]), (action_id, torch.tensor([-1]).to(device), torch.tensor([-1]).to(device))

        elif 1 <= int(action_id.cpu().numpy()) <= 7:
            action = action_space[int(action_id.cpu().numpy())]
            for object_id in object_prediction:
                obj = observations[list(observations.keys())[int(object_id.cpu().numpy())]]['name']
                index = list(observations.keys())[int(object_id.cpu().numpy())]
                if judge_1(action, obj, index, relationships, observations):
                    return '[{}] <{}> ({})'.format(action, obj, index), (action_id, object_id, torch.tensor([-1]).to(device))

        elif int(action_id.cpu().numpy()) >= 8:
            action = action_space[int(action_id.cpu().numpy())]
            for i in range(len(object_prediction)):
                obj1 = observations[list(observations.keys())[int(object_prediction[i].cpu().numpy())]]['name']
                index1 = list(observations.keys())[int(object_prediction[i].cpu().numpy())]
                for j in range(i + 1, len(object_prediction)):
                    obj2 = observations[list(observations.keys())[int(object_prediction[j].cpu().numpy())]]['name']
                    index2 = list(observations.keys())[int(object_prediction[j].cpu().numpy())]
                    if judge_2(action, index1, index2, relationships, observations):
                        return '[{}] <{}> ({}) <{}> ({})'.format(action, obj1, index1, obj2, index2), (action_id, object_prediction[i], object_prediction[j])

In [8]:
prediction

{'action': tensor([[0, 1, 2, 6, 7, 4, 5, 3]], device='cuda:0'),
 'char': tensor([[1, 0]], device='cuda:0'),
 'object1': tensor([[  0, 258, 321,  76,  74, 256,  32,  77,   1, 131,  75, 257,  31, 304,
          336,  39, 294,  92,  71, 288, 313,  68, 295, 316,   2, 325, 130, 390,
          260,  80, 296, 111, 114, 318,  38, 312, 123, 307, 330,  78, 112, 310,
           67, 132,  30,  33, 319, 133, 320, 337, 314,  57,   3, 140,  34,  56,
            4, 286, 261, 259, 378, 134, 332,  64,  69, 322, 391,  63,  81,  62,
          323, 386,  48, 327, 368,  65,  61, 338,  66, 287, 135, 334, 289, 324,
          331, 136,  60,  73, 380, 308, 315, 137,  82, 326,  49, 388,  72, 311,
          290, 379, 387, 348,  59, 333, 305,   6, 115, 122, 328,  83, 371, 293,
          292, 385,  40, 297, 298,  93, 369,  43,  58, 394, 389, 317, 349, 367,
          138,  52, 393,  70, 113,  79, 299, 392,  51, 370,  50,  54,  42,  41,
          124, 366, 139, 291,  55,  36, 129,   5, 377,  84, 262,  37, 306, 381,
 